### 검증 세트
테스트 세트를 사용하지 않으며 모델이 과대적합인지 과소적합인지 판단하기 어렵다.
테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 나누는 것이다.
이 데이터를 <strong>검증 세트</strong>라고 부른다.

In [2]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')


In [3]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state=42
)

In [5]:
# 훈련세트의 약 20 퍼센트를 또 나눔
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state= 42
)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01342463, 0.010216  , 0.01055717, 0.01063633, 0.01030946]), 'score_time': array([0.00136256, 0.00134254, 0.00124717, 0.00119829, 0.00125599]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠이 가능하다.

한 가지 주의할 점은 cross_validate 훈련 세트를 섞어 폴드를 나누지 ㅇ낳는다.
훈련 세트를 섞으려면 분할기 splitter를 사용해야한다.


In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


훈련세트를 섞은 후 10-폴드 교차 검증을 수행하려면

In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [11]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

0.0001 부터 0.0005 까지 0.0001 씩 커지는 5개의 값을 시도한다.


In [12]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42),params, n_jobs = -1)

In [13]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [14]:
dt = gs.best_estimator_

print(dt.score(train_input, train_target))

0.9615162593804117


In [15]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [16]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


여기서는 0.0001 값이 가장 좋은 값으로 선택되었다 . 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results에 저장되어있다.


In [18]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


첫 번째 값이 가장 크다. 수동으로 고르는 것보다 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱르를 추출한다.
그다음 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다.
이 값이 최상의 검증 점수를 만든 매개변수 조합이다.
gs.best_params 와 동일한지 확인하자

In [19]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 먼저 탐색할 매개변수 지정
2. 훈련세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다.
이조합은 그리드 서치 객체에 자동 저장된다.
3. 그리드 서치는 최상의 매개변수에서 전체 훈련세트를 사용해 최종 모델을 훈련한다.

결정 트리에서 min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정한다.
max_depth로 트리의 깊이를 제한하고 노드를 나누기위한 최소 샘플수도 고르자


In [21]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
          'max_depth':range(5,20,1),
          'min_samples_split':range(2,100,10)
          }

15 * 9 * 10 = 1350번 반복한다
1350 번 교차 검증을 실시한다.

In [24]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [26]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [27]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


일일히 매개변수를 일일이 바꿔가며 교차 검증을 수행하지 않고 원하는 매개변수 값을 나열하면 자동으로 교차 검증을 수행해서 최상의 매개 변수를 찾을 수 있다.

### 랜덤 서치
매개 변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다.
너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다.
이럴 때 랜덤 서치를 사용하면 좋다.

랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링

In [28]:
from scipy.stats import uniform, randint

싸이파이의 stats 서브 패키지에 있는 uniform 과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다.
ranint는 정수값을 뽑고, uniform은 실수값을 뽑는다. 0에서 10사이의 범위를 갖는 randint 객체를 만들고 10개의 숫자를 샘플링 해보자


In [29]:
rgen = randint(0,10)
rgen.rvs(10)

array([4, 0, 8, 7, 4, 9, 3, 2, 7, 4])

In [30]:
np.unique(rgen.rvs(1000),return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([101,  90,  90, 110,  98, 112,  88, 104, 100, 107]))

In [31]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.20079375, 0.22902976, 0.98174599, 0.83662952, 0.91192071,
       0.48022925, 0.93830733, 0.12598455, 0.84735159, 0.43007612])

In [43]:
params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'max_depth':randint(20,50),
    'min_samples_split':randint(2,25),
    'min_samples_leaf':randint(1,25)
}

min_inpurity_decrease 는 0.0001에서 0.001 사이의 실숫값을 샘플링한다.
샘플링횟수는 사이킷런의 랜덤 서치 클래스인 randomizedSearchCV 의 n_iter


In [44]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x799c26315600>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x799c263152a0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x799c25e7d000>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x799c25e7cd60>},
                   random_state=42)

In [45]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [46]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [47]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


테스트 세트는 최종 모델을 선택할 때 까지 사용하지 말아야한다.
테스트 세트를 사용하지 않고도 모델을 평가하려면 또 다른 세트가 필요하다.
이를 검증 세트라 부른다.
혹은 개발 세트라 부른다.
검증 세트는 훈련 세트 중 일불ㄹ 다시 덜어내고 만든다.

검증 세트가 크지 않다면 어떻게 데이터를 나누었는지에 따라 검증 점수가 들쭉 날쭉 할 것이다.
훈련한 모델의 성능을 안정적으로 평가하려면 검증 세트를 한 번 나누어 모델을 평가하는 것에 그치지 않고, 여러번 반복이 가능하다.
이를 교차검증이라 한다.

보통 훈련세트를 5등분 혹은 10등분 한다. 이 한 덩어리를 폴드라 한다.

교차검증을 사용해 다양한 하이퍼 파라미터를 탐색한다.
그리드 서치를 사용하면 편리하다.

매개변수 값이 수치형이고 특히 연속적인 실숫값이라면 싸이파이의 확률 분포 객체를 전달하여 특정 범위 내에서 지정된 횟수만큼 매개변수 후보 값을 샘플링하여 교차 검증 시도가 가능하다,
